In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-text
import tensorflow_text as text
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.7 MB/s 
     |████████████████████████████████| 588.3 MB 19 kB/s 
     |████████████████████████████████| 6.0 MB 57.0 MB/s 
     |████████████████████████████████| 1.7 MB 48.3 MB/s 
     |████████████████████████████████| 439 kB 75.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data_for_colab/tripadvisor_hotel_reviews.csv')

df = df[df.Rating != 3]

df['is_positive'] = (df['Rating'] >= 4).astype(int)

<ipython-input-3-3afab3355398>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_positive'] = (df['Rating'] >= 4).astype(int)


In [ ]:
df.drop(columns=['Rating'], inplace=True)

In [ ]:
df.columns = ['DATA_COLUMN', 'LABEL_COLUMN']
df

,DATA_COLUMN,LABEL_COLUMN
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1
5,love monaco staff husband stayed hotel crazy w...,1
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",1
20487,great location price view hotel great quick pl...,1
20488,"ok just looks nice modern outside, desk staff ...",0
20489,hotel theft ruined vacation hotel opened sept ...,0


In [ ]:
df['LABEL_COLUMN'].value_counts()

1    15093
0     3214
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_positive = df[df['LABEL_COLUMN']==1]

In [ ]:
df_negative = df[df['LABEL_COLUMN']==0]

In [ ]:
# Тестовая выборка

In [ ]:
n_test = (df_negative.shape[0] // 4) * 3 # в оригинале (df_negative.shape[0] // 4) * 3
df_negative_test = df_negative.tail(n_test)
n_test = (df_positive.shape[0] // 20) * 3 # в оригинале df_positive.shape[0] // 20) * 3
df_positive_test = df_positive.tail(n_test)

In [ ]:
df_positive_test

,DATA_COLUMN,LABEL_COLUMN
17565,wonderful hotel wonderful town unbeknownst man...,1
17566,wonderful wonderful wonderful just returned ye...,1
17567,"affinia 50 you'all, stayed affinia 50 week, li...",1
17568,just right good location walking distance bloo...,1
17569,loved place want thank outstanding service goi...,1
...,...,...
20480,great play stay stay loyal inn package deal ha...,1
20482,great choice wife chose best western quite bit...,1
20483,good bed clean convenient just night happy sta...,1
20486,"best kept secret 3rd time staying charm, not 5...",1


In [ ]:
df_balanced_test = pd.concat([df_negative_test, df_positive_test])

In [ ]:
df_balanced_test.sample(10)

,DATA_COLUMN,LABEL_COLUMN
17658,great hotel great location location right hear...,1
20202,checked night little spoiled disappointed da h...,0
10703,good location average hotel good points:1. cen...,0
17056,"n't bother stay door stayed week end november,...",0
15199,"not fans, reserved room mandarin oriental amer...",0
10537,"great location pity rooms, situation n't bette...",0
13898,service snarl date november 14 2008i stayed ma...,0
20034,"great value fell love beach, travelled boyfrie...",1
18111,pleasant stay hotel fantastic hotel 18 floors ...,1
20066,excellent value money went breezes 2 weeks 2 a...,1


In [ ]:
df_balanced_test['LABEL_COLUMN'].value_counts()

0    2409
1    2262
Name: LABEL_COLUMN, dtype: int64

In [ ]:
# Обучающая выборка

In [ ]:
n_train = df_negative.shape[0] // 4 # в оригианле df_negative.shape[0] // 4
df_negative_train = df_negative.head(n_train)
n_train = df_positive.shape[0] // 20 # в оригинале df_positive.shape[0] // 20
df_positive_train = df_positive.head(n_train)

In [ ]:
df_balanced_train = pd.concat([df_negative_train, df_positive_train])

In [ ]:
df_balanced_train['LABEL_COLUMN'].value_counts()

0    803
1    754
Name: LABEL_COLUMN, dtype: int64

In [ ]:
df_balanced_train.sample(10)

,DATA_COLUMN,LABEL_COLUMN
117,"best bar lobby meet friend year, pop elevator ...",1
2621,not scene conference did n't choose looking fo...,0
697,"n't hotels website pictures, hello dissapointe...",0
2725,star ghetto motel hi arrived ant recomended su...,0
1138,great hotel couples sydney australia stayed ca...,1
1058,b islanders loved ws 30th wedding anniversary ...,1
1468,"n't make mistake, online reservation price cha...",0
102,absolutely charming good value stayed numerous...,1
476,"definitely stay, stay hampton inn travel work ...",1
3165,not place great group guys enjoy golf package ...,0


In [ ]:
X_train = df_balanced_train['DATA_COLUMN'].squeeze()
y_train = df_balanced_train['LABEL_COLUMN'].squeeze()

In [ ]:
X_test = df_balanced_test['DATA_COLUMN'].squeeze()
y_test = df_balanced_test['LABEL_COLUMN'].squeeze()

In [ ]:
tn_bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3') # uses same thing as bert


In [ ]:
tn_bert_encoder = hub.KerasLayer("https://tfhub.dev/google/tn_bert/1", trainable=False)

In [ ]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = tn_bert_preprocess(text_input)
outputs = tn_bert_encoder(preprocessed_text)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)


In [ ]:
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
49/49 [==============================] - 25s 259ms/step - loss: 0.5894 - accuracy: 0.7611 - precision: 0.7567 - recall: 0.7467
Epoch 2/10
49/49 [==============================] - 13s 260ms/step - loss: 0.4412 - accuracy: 0.8735 - precision: 0.8789 - recall: 0.8568
Epoch 3/10
49/49 [==============================] - 13s 263ms/step - loss: 0.3741 - accuracy: 0.8850 - precision: 0.8808 - recall: 0.8820
Epoch 4/10
49/49 [==============================] - 13s 266ms/step - loss: 0.3372 - accuracy: 0.8838 - precision: 0.8846 - recall: 0.8740
Epoch 5/10
49/49 [==============================] - 13s 267ms/step - loss: 0.3093 - accuracy: 0.8915 - precision: 0.8864 - recall: 0.8899
Epoch 6/10
49/49 [==============================] - 13s 268ms/step - loss: 0.2950 - accuracy: 0.8927 - precision: 0.8940 - recall: 0.8833
Epoch 7/10
49/49 [==============================] - 13s 269ms/step - loss: 0.2813 - accuracy: 0.9011 - precision: 0.9054 - recall: 0.8886
Epoch 8/10
49/49 [================

In [30]:
y_predicted = model.predict(X_test)
y_predicted


146/146 [==============================] - 39s 266ms/step


array([[0.28195044],
       [0.08911375],
       [0.11857634],
       ...,
       [0.7959341 ],
       [0.9079714 ],
       [0.9846406 ]], dtype=float32)

In [31]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [32]:
y_test

4133     0
4134     0
4135     0
4136     0
4140     0
        ..
20480    1
20482    1
20483    1
20486    1
20487    1
Name: LABEL_COLUMN, Length: 4671, dtype: int64

In [33]:
accuracy_score(y_test, y_predicted)

0.8644829800899165

In [34]:
precision_score(y_test, y_predicted)


0.8800746616892208

In [35]:
recall_score(y_test, y_predicted)

0.8337754199823165

In [36]:
f1_score(y_test, y_predicted)

0.856299659477866

In [37]:
df_results_for_reviews_on_hotels = pd.DataFrame(columns=['accuracy', 'precision', 'recall', 'f1_score'])

In [38]:
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'accuracy'] = accuracy_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'precision'] = precision_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'recall'] = recall_score(y_test, y_predicted)
df_results_for_reviews_on_hotels.loc['reviews on hotels', 'f1_score'] =  f1_score(y_test, y_predicted)


In [39]:
saved_model_path = '/content/drive/MyDrive/data_for_colab/tn_bert_trained_on_hotel_reviews_02_january'

In [40]:
print(saved_model_path)

/content/drive/MyDrive/data_for_colab/tn_bert_trained_on_hotel_reviews_02_january


In [41]:
model.save(saved_model_path, include_optimizer=True) 

In [42]:
df_results_for_reviews_on_hotels


,accuracy,precision,recall,f1_score
reviews on hotels,0.864483,0.880075,0.833775,0.8563
